# Get Base Climatology

This notebook automatically discovers the latest LDAS forecast initialization and builds the base climatology products by averaging the historical December‑01 hindcasts tied to that run. It produces deterministic monthly means for each surface variable and derives hydrologic zonal averages for the active area of interest.

Surface variable units:
- `Rainf_tavg`: mm/day
- `Qair_f_tavg`: g/kg
- `Qs_tavg`: mm/day
- `Evap_tavg`: mm/day
- `Tair_f_tavg`: degree Celsius
- `SoilMoist_inst`: m^3 m-3
- `SoilTemp_inst`: degree Celsius
- `Streamflow_tavg`: m^3/s


In [ ]:
import xarray as xr
import regionmask
import geopandas as gpd
import xarray as xr
import os
import pandas as pd
import re
from tqdm import tqdm
from datetime import datetime
from pathlib import Path

## Global variables
list_of_variables = {'Rainf_tavg' : 'Precipitation', 
                    'Qair_f_tavg' : 'Specific humidity',
                    'Qs_tavg':'Surface runoff',
                    'Evap_tavg':'Evapotranspiration',
                    'Tair_f_tavg':'Air temperature',
                    'SoilMoist_inst': 'Soil moisture',
                    'SoilTemp_inst': 'Soil temperature',
                    'Streamflow_tavg': 'Stream flow'}

geodataframe_path = "https://raw.githubusercontent.com/blackteacatsu/" \
"spring_2024_envs_research_amazon_ldas/main/resources/hybas_sa_lev05_areaofstudy.geojson"


# Load geodataframe and get all PFAF_Ids_forecast
geodataframe = gpd.read_file(geodataframe_path)
pfaf_ids_aoi = geodataframe.PFAF_ID.unique()

output_dir = "get_zonal_averages_climatology_csv"
os.makedirs(output_dir, exist_ok=True)
soilmoist_levels = [0,1,2,3]
soiltemp_levels = [0,1,2,3]

Recognize latest forecast and pick out corresponding hindcast data

In [3]:
# --- helpers ---
_MONTHS = {m: i+1 for i, m in enumerate(
    ["jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
)}

_PATTERNS = [
    re.compile(r'^ldas_fcst_(\d{4})_([a-z]{3})(\d{2})\.nc$', re.I),  # ldas_fcst_2024_dec01.nc
    re.compile(r'^ldas_fcst_(\d{4})(\d{2})(\d{2})\.nc$', re.I),      # ldas_fcst_20241201.nc
]

def _parse_date_from_name(name: str) -> datetime | None:
    for pat in _PATTERNS:
        m = pat.match(name)
        if not m:
            continue
        if pat is _PATTERNS[0]:
            y = int(m.group(1))
            mon = _MONTHS.get(m.group(2).lower())
            d = int(m.group(3))
        else:
            y, mon, d = int(m.group(1)), int(m.group(2)), int(m.group(3))
        if mon and 1 <= mon <= 12 and 1 <= d <= 31:
            return datetime(y, mon, d)
    return None

def forecast_init_datetime(fpath: str) -> datetime:
    dt = _parse_date_from_name(Path(fpath).name)
    if dt is None:
        raise ValueError(f"Unrecognized forecast filename format: {fpath}")
    return dt

# --- main: forecast = latest by encoded date; hindcasts = existing Dec-01 files from prior years ---
def split_forecast_and_dec_hindcasts(
    dir_path: str,
    prefix: str = "ldas_fcst_",
    recursive: bool = False
):
    base = Path(dir_path)
    if not base.is_dir():
        raise NotADirectoryError(f"Not a directory: {dir_path}")

    pattern = "**/*.nc" if recursive else "*.nc"
    items = []
    for p in base.glob(pattern):
        if not p.is_file():
            continue
        name = p.name
        if not name.startswith(prefix) or not name.endswith(".nc"):
            continue
        dt = _parse_date_from_name(name)
        if dt is None:
            continue
        items.append((dt, p.stat().st_mtime, name, p))

    if not items:
        raise FileNotFoundError(f"No matching .nc files found in {dir_path} (prefix='{prefix}')")

    # latest by (date, mtime, name)
    items.sort(key=lambda t: (t[0], t[1], t[2]))
    forecast_path = items[-1][3]
    forecast_dt = items[-1][0]

    # Hindcasts = existing Dec-01 files from earlier years only
    hindcasts = [
        p for (dt, _, _, p) in items
        if dt.year < forecast_dt.year and dt.month == 12 and dt.day == 1
    ]
    # Sort hindcasts by year ascending (oldest → newest)
    hindcasts.sort(key=lambda p: _parse_date_from_name(p.name))

    return str(forecast_path), [str(p) for p in hindcasts], forecast_dt

# --- usage ---
surface_model_file_path = r"/mnt/vast/prakrut/backup/malaria_amazon/amazon_forecast"

forecast_file, hindcast_files, f_dt = split_forecast_and_dec_hindcasts(surface_model_file_path)

print("Forecast file:", forecast_file)          # e.g., .../ldas_fcst_2024_dec01.nc
print("Hindcasts   :", len(hindcast_files))     # only Dec-01 of prior years
print("Init date   :", f_dt)                    # 2024-12-01 00:00:00

#print(hindcast_files)
# If you still want the 'YYYY_mon' tag:
initialization_date = f"{f_dt.year}_{f_dt.strftime('%b').lower()}"  # '2024_dec'
print("Forecast initialization date:", initialization_date)


Forecast file: /mnt/vast/prakrut/backup/malaria_amazon/amazon_forecast/ldas_fcst_2024_dec01.nc
Hindcasts   : 23
Init date   : 2024-12-01 00:00:00
Forecast initialization date: 2024_dec


## Functions

In [5]:
# Read gridded data array and find coordinates variable
def get_standard_coordinates(dataset: xr.Dataset, lon_names=None, lat_names=None, time_names=None):
    """
    Retrieve longitude, latitude, and time variables from an xarray dataset,
    accommodating different naming conventions.

    Parameters:
        dataset (xr.DataArray): The xarray data array to search.
        lon_names (list): List of possible names for longitude (default: common names).
        lat_names (list): List of possible names for latitude (default: common names).
        time_names (list): List of possible names for time (default: common names).

    Returns:
        tuple: Longitude, latitude, and time variables.

    Raises:
        AttributeError: If any of the required variables are not found.
    """

    lon_names = lon_names or ["east_west", "lon", "longitude"]
    lat_names = lat_names or ["north_south", "lat", "latitude"]
    time_names = time_names or ["time", "month", "date"]

    def find_variable(dataset, possible_names):
        for name in possible_names:
            if name in dataset.dims:
                return dataset[name]
        raise AttributeError(f"None of the variable names {possible_names} found in the dataset.")

    # Try to find longitude, latitude, and time variables
    lon = find_variable(dataset, lon_names)
    lat = find_variable(dataset, lat_names)
    time = find_variable(dataset, time_names)

    return lon, lat, time

In [6]:
# Read climatology data files & compute monthly averages
def initialize_climatology(hindcast_data_file_path, variable):
    """
    Initialize climatology for the given variable from hindcast data.
    """
    climatology = xr.open_mfdataset(hindcast_data_file_path)[variable]
    climatology = climatology.groupby('time.month').mean(dim='time')
    #if variable == 'Stream_flow': # for streamflow, we want max not mean
        #climatology = climatology.groupby('time.month').max(dim='time') 
    return climatology


## Executable

Trim hindcast data to the variables listed, and store each variables separately as `.nc` an intermediate step.

In [7]:
for variable in tqdm(list_of_variables.keys()):
    
    print(list_of_variables.get(variable))
    climatology = initialize_climatology(hindcast_files, variable)

    #print(climatology.name)

    file_savepath = './get_deterministic_climatology/deterministic_' + initialization_date.replace('-','_')
    
    climatology.to_netcdf(file_savepath + '_climatology_' + str(variable) + '.nc')

    print('Saved climatology values for ' + str(list_of_variables.get(variable)) + '!')

  0%|          | 0/8 [00:00<?, ?it/s]

Average precipitation


 12%|█▎        | 1/8 [00:15<01:46, 15.28s/it]

Saved climatology values for Average precipitation!
Specific humidity


 25%|██▌       | 2/8 [00:29<01:27, 14.65s/it]

...

Saved climatology values for Soil moisture!
Soil temperature


 88%|████████▊ | 7/8 [02:05<00:21, 21.78s/it]

Saved climatology values for Soil temperature!
Stream flow


100%|██████████| 8/8 [02:18<00:00, 17.28s/it]

Saved climatology values for Stream flow!


Calculate zonal averaged climatology,  final output is store in `.csv` format.

In [8]:
ds_climatology = xr.open_mfdataset(rf'get_deterministic_climatology/*.nc')

lon, lat, months = get_standard_coordinates(ds_climatology)

output_dir = "get_zonal_averages_climatology_csv"
os.makedirs(output_dir, exist_ok=True)

for pfaf_id in tqdm(pfaf_ids_aoi): # Iterate over each PFAF_ID
    print(f'Processing PFAF_ID: {pfaf_id}')
    aoi = geodataframe[geodataframe.PFAF_ID == pfaf_id] 

    if aoi.empty:
        continue
    aoi_mask = regionmask.mask_3D_geopandas(aoi, lon, lat) # Create AOI mask

    records = [] # Initialize records list
    # Iterate over time and ensemble dimensions
    for t in ds_climatology['month'].values: 
        row = {'month': t, 'pfaf_id': pfaf_id} # Initialize row with time, and PFAF_ID
        for var in list_of_variables.keys(): # Iterate over each variable
            # Check if variable is SoilMoist_inst or SoilTemp_inst to handle levels
            if var == 'SoilMoist_inst': 
                for lvl in soilmoist_levels: 
                    col = f'SoilMoist_inst_lvl_{lvl}' # Create column name for soil moisture levels
                    if 'SoilMoist_profiles' in ds_climatology[var].dims:
                        data = ds_climatology[var].sel(month=t, SoilMoist_profiles=lvl)
                        if 'ensemble' in data.dims:
                            data = data.mean(dim='ensemble') # Average over ensemble if exists
                        masked = data.where(aoi_mask)
                        row[col] = masked.mean(dim=['lat','lon'], skipna=True).values.item()
                    else:
                        row[col] = None
            elif var == 'SoilTemp_inst':
                for lvl in soiltemp_levels:
                    col = f'SoilTemp_inst_lvl_{lvl}'
                    if 'SoilTemp_profiles' in ds_climatology[var].dims:
                        data = ds_climatology[var].sel(month=t, SoilTemp_profiles=lvl)
                        if 'ensemble' in data.dims:
                            data = data.mean(dim='ensemble') # Average over ensemble if exists
                        masked = data.where(aoi_mask)
                        row[col] = masked.mean(dim=['lat','lon'], skipna=True).values.item()
                    else:
                        row[col] = None
            elif var == 'Streamflow_tavg':
                col = 'Streamflow_tavg'
                if var in ds_climatology.variables:
                    data = ds_climatology[var].sel(month=t)
                    if 'ensemble' in data.dims:
                        data = data.mean(dim='ensemble') # Average over ensemble if exists
                    masked = data.where(aoi_mask)
                    row[col] = masked.max(dim=['lat','lon'], skipna=True).values.item() # Use max for streamflow
                else:
                    row[col] = None
            
            else:
                if var in ds_climatology.variables:
                    data = ds_climatology[var].sel(month=t)
                    if 'ensemble' in data.dims:
                        data = data.mean(dim='ensemble') # Average over ensemble if exists
                    masked = data.where(aoi_mask)
                    row[var] = masked.mean(dim=['lat','lon'], skipna=True).values.item()
                else:
                    row[var] = None
        records.append(row)
    df = pd.DataFrame(records)
    out_csv = os.path.join(output_dir, f"zonal_climatology_pfaf_{pfaf_id}.csv")
    df.to_csv(out_csv, index=False)
    print(f"Saved: {out_csv}")

  0%|          | 0/151 [00:00<?, ?it/s]

Processing PFAF_ID: 61581


  1%|          | 1/151 [00:03<07:54,  3.17s/it]

Saved: get_zonal_averages_climatology_csv/zonal_climatology_pfaf_61581.csv
Processing PFAF_ID: 61593


  1%|▏         | 2/151 [00:06<07:35,  3.06s/it]

...

Saved: get_zonal_averages_climatology_csv/zonal_climatology_pfaf_67207.csv
Processing PFAF_ID: 67208


 99%|█████████▉| 150/151 [07:21<00:02,  2.96s/it]

Saved: get_zonal_averages_climatology_csv/zonal_climatology_pfaf_67208.csv
Processing PFAF_ID: 66405


100%|██████████| 151/151 [07:24<00:00,  2.94s/it]

Saved: get_zonal_averages_climatology_csv/zonal_climatology_pfaf_66405.csv
